In [1]:
import pyLDAvis.gensim_models as gensimvis
import nltk
import re
import pandas as pd
import string
import gensim

from nltk.corpus import stopwords
from openai import AzureOpenAI
from gensim import corpora, models
nltk.download('punkt')
nltk.download('stopwords')
indonesian_stopwords = set(stopwords.words('indonesian'))
english_stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
class Llm:
    def getContext(topics, keyword, best_num_topics_str):
        # Data dengan bobot kata-kata tertinggi untuk setiap topik
        data = [topics]

        # Mengolah data untuk mengambil kata-kata terbobot tertinggi
        topic_keywords = []
        for topic_data in data:
            topic_data = topic_data[0][1]  # Ambil data topik pertama (bobot tertinggi)
            topic_data = sorted(topic_data, key=lambda x: x[1], reverse=True)  # Urutkan berdasarkan bobot terbesar ke terkecil
            keywords = [word[0] for word in topic_data[:10]]  # Ambil 10 kata dengan bobot tertinggi
            topic_keywords.extend(keywords)

        # Create a prompt using the keywords
        role = "AI Linguistik"
        action = "menentukan kalimat dari beberapa topik berdasarkan dari kumpulan kata-kata hasil dari proses LDA"
        step = ("mempertimbangkan bobot setiap topik yang ada pada penomoran angka dalam merangkai kata-kata kunci "
                "menjadi kalimat yang padu untuk sebuah topik yang diperbincangkan di Twitter. "
                "Dengan menggunakan penomoran untuk setiap topik, ambil kata inti dari hasil LDA lalu susun menjadi "
                "kalimat yang padu dan mudah dipahami.")
        context = (f"Anda akan membahas tentang topik dengan kata-kata kunci berikut: \n\n"
                f"{' '.join(topic_keywords)} \n\n"
                f"dengan berbagai pandangan masyarakat terhadap topik tersebut.")
        example = ("Misalnya, jika hasil analisis LDA mengidentifikasi kata-kata kunci seperti 'ukt', 'mahal', 'banget', "
                "'jual', 'usia', 'kemendikbudristek', 'hoyong', 'maba', 'nilainilai', 'kemaren', Anda akan menciptakan "
                "kalimat seperti: 'Dalam pandangan beberapa masyarakat, kenaikan ukt dianggap sangat mahal banget dan ini "
                "mempengaruhi mahasiswa baru (maba) yang merasa terbebani, terutama mereka yang belum memiliki penghasilan "
                "dan masih berada pada usia muda. Beberapa orang bahkan bercanda tentang menjual barang-barang mereka untuk "
                "membayar biaya pendidikan. Selain itu, Kemendikbudristek diharapkan dapat menilai kembali kebijakan ini untuk "
                "mempertimbangkan nilai-nilai kesetaraan dalam akses pendidikan. Kemarin, isu ini semakin hangat diperbincangkan "
                "di media sosial.'")
        format_str = f"dengan format kalimat naratif berbentuk bullet point yang menggabungkan semua kata kunci tersebut dengan jumlah sesuai jumlah topik yang diberikan yaitu : {best_num_topics_str}."

        # Combine components into the final prompt
        prompt = (f"# RASCEF = Role + ( Action + Step + Context + Example ) + Format\n\n"
                f"Role: {role}\n"
                f"Action: {action}\n"
                f"Step: {step}\n"
                f"Context: {context}\n"
                f"Example: {example}\n"
                f"Format: {format_str}\n\n"
                f"Kata-kata kunci: {', '.join(topic_keywords)}")

        # Inisialisasi Azure OpenAI client
        client = AzureOpenAI(
            api_version="2023-05-15",
            azure_endpoint="https://chatbot-aic.openai.azure.com/",
            api_key="fd068a3036e34fe188a28392699ecc65",
        )

        response = client.chat.completions.create(
            model="aicdeploymodel",  # Ganti dengan nama deployment Anda
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3
        )

        
        # Extract the generated sentence from the response
        generated_sentence = response.choices[0].message.content.replace('\n', '<br/>').replace(' .', '.').strip()
        # generated_sentence = response
        # Print the generated sentence
        return generated_sentence

In [53]:
class Lda:
    def clusster(data, keyword, num_topics=5):

        def clean_tweet(tweet):
            tweet = str(tweet).lower()
            tweet = re.compile('https?://\S+|www\.\S+').sub(r'', tweet) # Remove hyperlinks

            if tweet.startswith("rt"): # Remove retweets (repetitions)
                i = tweet.find(':')
                if i != -1:
                    tweet = tweet[i+2:]
            
            tweet = re.compile('@[^\s]+').sub(r'', tweet) # Mentions 
            tweet = re.compile(r'#([^\s]+)').sub(r'\1', tweet) # Remove hashtags
            tweet = re.sub('@', 'at', tweet)
            tweet  = ''.join([char for char in tweet if char not in string.punctuation]) #Remove punctuation characters
            tweet = re.compile('[^A-Za-z]').sub(r' ', tweet) # Remove any other non-alphabet characters
            tweet = ' '.join([w for w in tweet.split() if w not in indonesian_stopwords]) #Remove stop words  
            # tweet = ' '.join([w for w in tweet.split() if w not in english_stopwords]) #Remove stop words  
            
            return tweet

        def remove_single_letter_words(text):
            text = re.sub(r'\b\w\b', '', text)
            
            hapus = ['id','amp','deh','tanyakanrl','dtype','length','sih','na','si','rj','lc','ar','oe','al','sm','ri','en','ar','mc','vt','rob','ny','dc','az','va','mkmk','nya','do','ye']
            words = text.split()

            # Memfilter kata-kata yang tidak ada dalam array yang akan dihapus
            kata_kata_tanpa_kata_yang_dihapus = [kata for kata in words if kata not in hapus]

            # Menggabungkan kata-kata yang tersisa menjadi kalimat baru
            kalimat_tanpa_kata_yang_dihapus = ' '.join(kata_kata_tanpa_kata_yang_dihapus)
            return kalimat_tanpa_kata_yang_dihapus

        data = pd.DataFrame(data)
        # data = data.apply(fix_contractions_id)
        data = data.apply(clean_tweet)
        data = data.apply(remove_single_letter_words)

        tokenized_documents = [doc.split() for doc in data]
        # print(tokenized_documents)

        # Membuat kamus (dictionary)
        dictionary = corpora.Dictionary(tokenized_documents)

        # Membuat corpus (representasi vektor dari dokumen)
        corpus = [dictionary.doc2bow(doc) for doc in tokenized_documents]

        # Clusstering with LDA
        lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=20, alpha=1, eta=1)

        topics = lda_model.show_topics(num_topics=num_topics, log=False, formatted=False)
        # return topics
        # Print the topics
        res = []
        for topic_id, topic in topics:
            res.append([word for word, _ in topic])
            print(topic)

        print(res)     
        
        # return topics
        return Llm.getContext(topics, keyword, num_topics)

        

<>:6: DeprecationWarning: invalid escape sequence \S
<>:13: DeprecationWarning: invalid escape sequence \s
<>:6: DeprecationWarning: invalid escape sequence \S
<>:13: DeprecationWarning: invalid escape sequence \s
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20428\132160428.py:6: DeprecationWarning: invalid escape sequence \S
  tweet = re.compile('https?://\S+|www\.\S+').sub(r'', tweet) # Remove hyperlinks
C:\Users\lenovo\AppData\Local\Temp\ipykernel_20428\132160428.py:13: DeprecationWarning: invalid escape sequence \s
  tweet = re.compile('@[^\s]+').sub(r'', tweet) # Mentions


In [65]:
csv_file_path = './dataset/dataset-testing/impor-susu-sapi-jan-mei.csv'
df = pd.read_csv(csv_file_path)

data = df['full_text']

topics = Lda.clusster(data, 'impor susu', 5)

[('sapi', 0.07466889), ('india', 0.052936353), ('import', 0.05293352), ('program', 0.042103276), ('susu', 0.04210202), ('impor', 0.031322222), ('wowo', 0.020705596), ('iga', 0.02070538), ('penyet', 0.020705316), ('hamburka', 0.020705175)]
[('sapi', 0.025287876), ('india', 0.025240202), ('import', 0.025237376), ('program', 0.025197256), ('susu', 0.025197096), ('impor', 0.02512407), ('kerbau', 0.024963653), ('subianto', 0.024962978), ('prabowo', 0.024962831), ('nomor', 0.02496282)]
[('sapi', 0.02530226), ('import', 0.02524727), ('india', 0.025246115), ('susu', 0.025200523), ('program', 0.025200121), ('impor', 0.025124934), ('goreng', 0.024963113), ('nomor', 0.024962967), ('subianto', 0.024962265), ('indomie', 0.024962135)]
[('sapi', 0.025283508), ('import', 0.025243348), ('india', 0.025235731), ('susu', 0.025196336), ('program', 0.025194792), ('impor', 0.025125528), ('ngeri', 0.024963744), ('pd', 0.024963576), ('goreng', 0.024963224), ('nomor', 0.024963131)]
[('sapi', 0.025280552), ('imp

In [66]:
# Replace HTML line breaks with actual newlines for better readability
topics = topics.replace('<br/>', '\n')

print(topics)

1. Dalam pandangan beberapa masyarakat, program import sapi dari India dianggap sebagai ancaman bagi peternak lokal dan industri susu dalam negeri.
2. Terdapat perdebatan tentang kebijakan impor susu yang dianggap merugikan peternak lokal dan menguntungkan impor dari luar negeri.
3. Beberapa orang terkesan dengan keberhasilan program impor wowo yang berhasil menarik minat masyarakat untuk mencoba hidangan baru seperti iga penyet dan hamburka.
4. Namun, ada juga yang menentang program impor wowo karena dianggap mengancam keberlangsungan hidangan tradisional Indonesia.
5. Dalam pandangan beberapa masyarakat, kebijakan impor susu dan program impor wowo harus dipertimbangkan dengan hati-hati agar tidak merugikan industri dalam negeri dan melestarikan kekayaan kuliner Indonesia.


In [3]:
import requests
import pandas as pd

# Fetching data from the provided URL
url = "http://topic-socialabs.unikomcodelabs.id/topic?keyword=anies&num_topics=5&num_tweets=500"
response = requests.get(url)
data = response.json()

# Extract context
context = data['data']['context']

# Extract documents_topic
documents_topic = data['data']['documents_topic']

# Create a DataFrame
df = pd.DataFrame(documents_topic)

# Add context to each row
df['context'] = context
# Saving the DataFrame to a CSV file
csv_file_path = './dataset/dataset-testing/topik-anies.csv'
df.to_csv(csv_file_path, index=False)

df


,conversation_id_str,created_at,favorite_count,full_text,id_str,lang,probability,quote_count,reply_count,retweet_count,topic,tweet_url,user_id_str,username,context
0,1722416236083200098,Fri Nov 10 23:49:21 +0000 2023,0,@M45Broo Gpp halu dulu 🤣 noh manusia uban data...,1723125723156947294,in,0.97057265,0,0,0,3,https://twitter.com/nusa_talk/status/172312572...,1571698388785336320,nusa_talk,Berikut adalah kalimat-kalimat utama untuk set...
1,1723109580073521507,Fri Nov 10 23:39:41 +0000 2023,1,@MinmonPS7 Terus anies harus berpasangan dgn s...,1723123292335431687,in,0.93176955,0,2,0,0,https://twitter.com/tanz_77/status/17231232923...,187094886,tanz_77,Berikut adalah kalimat-kalimat utama untuk set...
2,1722966988979683638,Fri Nov 10 23:37:43 +0000 2023,0,@Tan_Mar3M Jni pertanyaan jebakan.....klu berp...,1723122796518334479,in,0.9614479,0,0,0,2,https://twitter.com/BumiOfai/status/1723122796...,1710774321076207616,BumiOfai,Berikut adalah kalimat-kalimat utama untuk set...
3,1722889979721097379,Fri Nov 10 23:34:36 +0000 2023,0,@tempodotco Pak Anies Baswedan mengajarkan kit...,1723122011566923957,in,0.94221425,0,0,0,2,https://twitter.com/koponng/status/17231220115...,883660630816727040,koponng,Berikut adalah kalimat-kalimat utama untuk set...
4,1722889979721097379,Fri Nov 10 23:31:02 +0000 2023,0,@tempodotco @mengenyam AMIN mungkin meraih has...,1723121115512664185,in,0.9638936,0,0,0,1,https://twitter.com/sebeelum/status/1723121115...,886925187907596292,sebeelum,Berikut adalah kalimat-kalimat utama untuk set...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1722939439385837814,Fri Nov 10 11:29:07 +0000 2023,0,Mantan Menag Fachrul Razi Sebut Anies-Muhaimin...,1722939439385837814,in,0.72561145,1,0,0,4,https://twitter.com/jpnncom/status/17229394393...,79130206,jpnncom,Berikut adalah kalimat-kalimat utama untuk set...
996,1722939188331614355,Fri Nov 10 11:28:07 +0000 2023,0,Kemenangan Pak Anies akan menginspirasi genera...,1722939188331614355,in,0.540799,0,0,0,2,https://twitter.com/SriLamtiur/status/17229391...,1678989817106034690,SriLamtiur,Berikut adalah kalimat-kalimat utama untuk set...
997,1722939111982653520,Fri Nov 10 11:27:49 +0000 2023,0,Pemimpin yang berkomitmen untuk kemajuan bangs...,1722939111982653520,in,0.9423321,0,0,0,2,https://twitter.com/wsri786031/status/17229391...,1678033921181896704,wsri786031,Berikut adalah kalimat-kalimat utama untuk set...
998,1722842978543366491,Fri Nov 10 11:27:09 +0000 2023,15,@andre_rosiade Semoga menjadi amal kebaikan da...,1722938945691095497,in,0.93272656,0,0,0,2,https://twitter.com/Saerudin16/status/17229389...,1316456812099432449,Saerudin16,Berikut adalah kalimat-kalimat utama untuk set...
